In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

> # **Ucitavanje skupova**

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

test = test.iloc[:,1:]

> # **Optimizacija podataka**





In [ ]:
# training set podijeljen na oznake (labels) i značajke (features)
X_train = train.iloc[:,1:]
y_train = train.iloc[:,0]

In [ ]:
# Promjena dimenzija slika
X_train = X_train.values.reshape(-1, 28, 28, 1)/255.
test = test.values.reshape(-1, 28, 28, 1)/255.

y_train = to_categorical(y_train, 16)

> # **Augmentacija podataka**



In [ ]:
datagen = ImageDataGenerator(
            rotation_range=10,
            width_shift_range=0.1,
            height_shift_range=0.1,
            zoom_range=0.1,
            shear_range=0.1
            )



> # **Definiranje modela**



In [ ]:
def create_model():
    model = Sequential()

    model.add(Conv2D(32, (5,5), padding='same', input_shape=X_train.shape[1:], activation='relu'))
    model.add(Conv2D(32, (5,5), padding='same', activation='relu'))
    model.add(MaxPool2D(2,2))

    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(MaxPool2D(2,2))

    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(16, activation='softmax'))
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

> # **Treniranje**


In [ ]:
EPOCHS = 20
BATCH_SIZE = 2
ENSEMBLES = 10
results = np.zeros((test.shape[0],16))
histories = []

callback_list = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.25, patience=2)
]

for i in range(ENSEMBLES):
    # podjela na training set i validation set
    X_train_tmp, X_val, y_train_tmp, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=i)
    # rad s modelom
    model = create_model()

    history = model.fit(datagen.flow(X_train_tmp, y_train_tmp, batch_size=BATCH_SIZE),
                   epochs=EPOCHS,
                   callbacks=callback_list,
                   validation_data=(X_val, y_val),
                   steps_per_epoch=X_train_tmp.shape[0] // BATCH_SIZE)
    # spremanje rezultata
    histories.append(history)
    results += model.predict(test)

> # **Kreiranje dokumenta s rezultatima**



In [ ]:
results = np.argmax(results, axis=1)
results = pd.Series(results, name='Label')
final = pd.concat([pd.Series(range(1,28001), name='ImageID'), results], axis=1)
final.to_csv('final.csv', index=False)